MENGGUNAKAN COLAB UNTUK MENTRAIN
MENGGUNAKAN GDOWN UNTUK DATASET

akurasi kecil kemungkinan karna dataset ada yang korup, tidak bisa dibuka, dan rusak menyebabkan akurasi yang kecil

In [ ]:
!pip install gdown facenet-pytorch timm scikit-learn

import os, zipfile

# === 1. Download ZIP dataset dari Google Drive ===
FILE_ID = "1tDo2zQC_1ZKY8aMYaalgr6nxRmBcRxaO"  # ID dari link yang kamu kirim
ZIP_PATH = "/content/trainset.zip"

!gdown --id {FILE_ID} -O {ZIP_PATH}

# === 2. Extract ZIP ke folder /content/dataset ===
EXTRACT_ROOT = "/content/dataset"
os.makedirs(EXTRACT_ROOT, exist_ok=True)

with zipfile.ZipFile(ZIP_PATH, 'r') as z:
    z.extractall(EXTRACT_ROOT)

print("Isi /content/dataset:", os.listdir(EXTRACT_ROOT))

# === 3. Set data_dir ke folder Train ===
data_dir = os.path.join(EXTRACT_ROOT, "Train")
print("Data train di:", data_dir)
print("Contoh isi train:", os.listdir(data_dir)[:10])


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1tDo2zQC_1ZKY8aMYaalgr6nxRmBcRxaO
From (redirected): https://drive.google.com/uc?id=1tDo2zQC_1ZKY8aMYaalgr6nxRmBcRxaO&confirm=t&uuid=f9c5d0bd-9499-47c3-bf35-6f1931a25fb5
To: /content/trainset.zip
100% 428M/428M [00:06<00:00, 67.5MB/s]
Isi /content/dataset: ['__MACOSX', 'Train']
Data train di: /content/dataset/Train
Contoh isi train: ['Bezalel Samuel Manik', 'hayyatul fajri', '.DS_Store', 'Arkan Hariz Chandrawinata Liem', 'Fajrul Ramadhana Aqsa', 'Mychael Daniel N', 'Kenneth Austin Wijaya', 'Raditya Erza Farandi', 'Zefanya Danovanta Tarigan', 'JP. Rafi Radiktya Arkan. R. AZ']


In [34]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from facenet_pytorch import MTCNN
from PIL import Image
import timm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Device: {device}")

# Buat folder models jika belum ada
models_dir = os.path.join(os.getcwd(), 'models')
os.makedirs(models_dir, exist_ok=True)
os.environ['TORCH_HOME'] = models_dir

# Load MTCNN untuk face detection & cropping
print("\n📸 Loading MTCNN...")
mtcnn = MTCNN(
    image_size=160,
    margin=20,
    device=device,
    keep_all=False,
    post_process=True
)
print("✅ MTCNN loaded!")

# Load ResNet50 untuk feature extraction (Model 1)
print("\n🧠 Loading ResNet50...")
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
resnet = nn.Sequential(*list(resnet.children())[:-1])  # Remove FC layer
resnet.eval()
resnet.to(device)
print("✅ ResNet50 loaded!")

# Transform untuk ResNet50
transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load DeiT-Small / ViT (Model 2)
print("\n🧠 Loading DeiT-Small (Vision Transformer)...")
vit = timm.create_model("deit_small_patch16_224", pretrained=True)
vit.reset_classifier(0)  # Buang FC layer → output embedding 384 dimensi
vit.eval()
vit.to(device)
print("✅ DeiT-Small loaded!")

# Transform untuk ViT (dengan augmentasi)
transform_vit = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),            # flip kiri-kanan
    transforms.RandomRotation(10),               # rotasi kecil ±10°
    transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.02
    ),                                           # variasi pencahayaan/warna
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],              # pakai ImageNet stats
        std=[0.229, 0.224, 0.225],
    ),
])


# Fungsi crop wajah (dipakai kedua model)
def crop_face(image_path):
    """
    Baca gambar → deteksi wajah → crop dengan MTCNN
    Return tensor (3,160,160) atau None kalau gagal.
    """
    try:
        img = Image.open(image_path).convert("RGB")
        face_tensor = mtcnn(img)

        if face_tensor is None:
            return None

        return face_tensor
    except Exception as e:
        print(f"⚠️ Error memproses {image_path}: {e}")
        return None

print("✅ crop_face() siap digunakan!")


🖥️  Device: cuda

📸 Loading MTCNN...
✅ MTCNN loaded!

🧠 Loading ResNet50...
✅ ResNet50 loaded!

🧠 Loading DeiT-Small (Vision Transformer)...
✅ DeiT-Small loaded!
✅ crop_face() siap digunakan!


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# ----- Encode label ke angka (0..N-1) -----
le = LabelEncoder()
y_encoded = le.fit_transform(filtered_labels_vit)

# Split train/val (pakai test_size yang tadi sudah dihitung)
X_train_vit, X_val_vit, y_train_vit, y_val_vit = train_test_split(
    filtered_embeddings_vit,
    y_encoded,
    test_size=test_size_vit,   # pakai test_size_vit yang sudah kamu hitung
    random_state=42,
    stratify=y_encoded
)

print("X_train:", X_train_vit.shape, "X_val:", X_val_vit.shape)

# ----- Latih classifier logistic regression -----
clf = LogisticRegression(
    max_iter=5000,
    n_jobs=-1,
    multi_class="multinomial"
)
clf.fit(X_train_vit, y_train_vit)

# ----- Evaluasi -----
y_pred = clf.predict(X_val_vit)

acc = accuracy_score(y_val_vit, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_val_vit, y_pred, average="weighted", zero_division=0
)
cm = confusion_matrix(y_val_vit, y_pred)

print("\n📊 Classification report (LogReg + DeiT embedding):")
print(classification_report(y_val_vit, y_pred, target_names=le.classes_, zero_division=0))

print(f"\nAccuracy: {acc:.4f}")
print(f"Weighted F1: {f1:.4f}")

# ----- Simpan model -----
model_data_vit_logreg = {
    "model_type": "DeiT-Small + LogisticRegression",
    "label_encoder": le,
    "classifier": clf,
    "validation_results": {
        "accuracy": acc,
        "precision_weighted": prec,
        "recall_weighted": rec,
        "f1_weighted": f1,
        "confusion_matrix": cm,
    }
}

vit_logreg_path = os.path.join(models_dir, "face_recognition_deit_small_logreg.pkl")
with open(vit_logreg_path, "wb") as f:
    pickle.dump(model_data_vit_logreg, f)

print(f"💾 Model DeiT-Small + LogisticRegression disimpan di: {vit_logreg_path}")


X_train: (175, 384) X_val: (66, 384)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



📊 Classification report (LogReg + DeiT embedding):
                                 precision    recall  f1-score   support

             Nasya Aulia Efendi       0.00      0.00      0.00         1
           Abraham Ganda Napitu       0.00      0.00      0.00         1
       Abu Bakar Siddiq Siregar       1.00      1.00      1.00         1
             Ahmad Faqih Hasani       0.00      0.00      0.00         1
                        Alfajar       1.00      1.00      1.00         1
            Alief Fathur Rahman       0.00      0.00      0.00         1
 Arkan Hariz Chandrawinata Liem       1.00      1.00      1.00         1
               Bayu Ega Ferdana       0.00      0.00      0.00         1
          Bayu Prameswara Haris       0.00      0.00      0.00         1
           Bezalel Samuel Manik       0.50      1.00      0.67         1
           Bintang Fikri Fauzan       0.00      0.00      0.00         1
              Boy Sandro Sigiro       0.00      0.00      0.00         

In [36]:
from collections import Counter

print("\n🚀 Membuat embedding DeiT-Small...")
vit_embeddings, vit_labels = build_embeddings(
    data_dir, extract_embedding_vit, desc="DeiT-Small embeddings"
)

# 🔧 Filter kelas yang punya sampel < 2
label_counts_vit = Counter(vit_labels)
print("\n📌 Jumlah sampel per kelas (ViT, sebelum filter):")
print(label_counts_vit)

filtered_embeddings_vit = []
filtered_labels_vit = []

for emb, lbl in zip(vit_embeddings, vit_labels):
    if label_counts_vit[lbl] >= 2:
        filtered_embeddings_vit.append(emb)
        filtered_labels_vit.append(lbl)

filtered_embeddings_vit = np.array(filtered_embeddings_vit)

num_samples_vit = len(filtered_labels_vit)
num_classes_vit = len(set(filtered_labels_vit))

print("\n✅ Setelah filter kelas < 2 sampel (ViT):")
print("Shape embeddings:", filtered_embeddings_vit.shape)
print("Total label:", num_samples_vit)
print("Total kelas:", num_classes_vit)

# 🔧 Hitung test_size minimal agar n_test >= n_classes
min_test_size_vit = num_classes_vit / num_samples_vit + 1e-6
test_size_vit = max(0.2, min_test_size_vit)

print(f"👉 test_size ViT diset ke: {test_size_vit:.3f}")

# Split train / val
X_train_vit, X_val_vit, y_train_vit, y_val_vit = train_test_split(
    filtered_embeddings_vit,
    filtered_labels_vit,
    test_size=test_size_vit,
    random_state=42,
    stratify=filtered_labels_vit
)

print("\n🔍 Validasi DeiT-Small ...")
results_vit = validate_model(X_train_vit, y_train_vit, X_val_vit, y_val_vit)

# Simpan model DeiT-Small
model_data_vit = {
    "model_type": "DeiT-Small",
    "train_embeddings": X_train_vit,
    "train_labels": y_train_vit,
    "val_labels": y_val_vit,
    "validation_results": results_vit,
}

vit_path = os.path.join(models_dir, "face_recognition_deit_small.pkl")
with open(vit_path, "wb") as f:
    pickle.dump(model_data_vit, f)

print(f"💾 Model DeiT-Small disimpan di: {vit_path}")



🚀 Membuat embedding DeiT-Small...


DeiT-Small embeddings:  81%|████████▏ | 57/70 [01:36<00:30,  2.38s/it]

⚠️ Error memproses /content/dataset/Train/Martua Kevin A.M.H.Lubis/IMG_20241208_223409_653 - MARTUA KEVIN ANDREAS MUAL H LUBIS.jpg: cannot identify image file '/content/dataset/Train/Martua Kevin A.M.H.Lubis/IMG_20241208_223409_653 - MARTUA KEVIN ANDREAS MUAL H LUBIS.jpg'


DeiT-Small embeddings:  91%|█████████▏| 64/70 [01:45<00:09,  1.61s/it]

⚠️ Error memproses /content/dataset/Train/Lois Novel E Gurning/foto 3 - LOIS NOVEL E. GURNING.png: cannot identify image file '/content/dataset/Train/Lois Novel E Gurning/foto 3 - LOIS NOVEL E. GURNING.png'
⚠️ Error memproses /content/dataset/Train/Lois Novel E Gurning/foto 4 - LOIS NOVEL E. GURNING.png: cannot identify image file '/content/dataset/Train/Lois Novel E Gurning/foto 4 - LOIS NOVEL E. GURNING.png'


DeiT-Small embeddings: 100%|██████████| 70/70 [01:54<00:00,  1.63s/it]

✅ DeiT-Small embeddings selesai. Shape embeddings: (243, 384), labels: 243

📌 Jumlah sampel per kelas (ViT, sebelum filter):
Counter({'Joshua Palti Sinaga': 5, 'Bezalel Samuel Manik': 4, 'hayyatul fajri': 4, 'Arkan Hariz Chandrawinata Liem': 4, 'Fajrul Ramadhana Aqsa': 4, 'Kenneth Austin Wijaya': 4, 'Zefanya Danovanta Tarigan': 4, 'JP. Rafi Radiktya Arkan. R. AZ': 4, 'Dyo Dwi Carol Bukit': 4, 'Ikhsannudin Lathief': 4, 'Joyapul Hanscalvin Panjaitan': 4, 'Fayyadh Abdillah': 4, 'Gabriella Natalya Rumapea': 4, 'Dwi Arthur Revangga': 4, 'Zaky Ahmad Makarim': 4, 'Royfran Roger Valentino': 4, 'Desty Ananta Purba': 4, 'Abu Bakar Siddiq Siregar': 4, 'Femmy Aprillia Putri': 4, 'Bayu Ega Ferdana': 4, 'Rahmat Aldi Nasda': 4, 'Ichsan Kuntadi Baskara': 4, 'Falih Dzakwan Zuhdi': 4, 'Randy Hendriyawan': 4, 'Joy Daniella V': 4, 'Garland Wijaya': 4, 'Reynaldi Cristian Simamora': 4, 'Zakhi algifari': 4, 'Ahmad Faqih Hasani': 4, 'Dimas Azi Rajab Aizar': 4, 'Freddy Harahap': 4, 'Alfajar': 4, 'Rayhan Fatih 

In [39]:
# ================================
# 1. IMPORT & CONFIG
# ================================
import os
import numpy as np
from collections import Counter
from PIL import Image

import torch
import torch.nn as nn
from torchvision import models, transforms
from torchvision.transforms.functional import to_pil_image

from facenet_pytorch import MTCNN

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report
)

import pickle

# ----------------
# Device & folder
# ----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Device: {device}")

# Folder models (untuk simpan .pkl)
models_dir = os.path.join(os.getcwd(), "models")
os.makedirs(models_dir, exist_ok=True)
os.environ["TORCH_HOME"] = models_dir

# Path dataset (ubah kalau beda)
data_dir = "/content/dataset/Train"   # <--- SESUAIKAN DENGAN STRUKTUR PUNYA KAMU
print(f"📁 Data dir: {data_dir}")

valid_ext = (".jpg", ".jpeg", ".png")


# ================================
# 2. MTCNN UNTUK FACE DETECTION
# ================================
print("\n📸 Loading MTCNN...")
mtcnn = MTCNN(
    image_size=160,
    margin=20,
    device=device,
    keep_all=False,
    post_process=True
)
print("✅ MTCNN loaded!")


# ================================
# 3. LOAD RESNET50 + TRANSFORM
# ================================
print("\n🧠 Loading ResNet50 (ImageNet pretrained)...")
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
# Jadikan layer terakhir (FC) = Identity, supaya output = embedding 2048-dim
resnet.fc = nn.Identity()
resnet.eval().to(device)
print("✅ ResNet50 loaded!")

transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225],   # ImageNet std
    ),
])


# ================================
# 4. FUNGSI CROP FACE & EMBEDDING
# ================================
def crop_face(img_path):
    """
    Baca gambar dari path, deteksi wajah pakai MTCNN,
    balikan tensor wajah (C,H,W) atau None kalau gagal.
    """
    try:
        img = Image.open(img_path).convert("RGB")
    except Exception as e:
        print(f"⚠️ Gagal buka gambar {img_path}: {e}")
        return None

    try:
        face, prob = mtcnn(img, return_prob=True)
    except Exception as e:
        print(f"⚠️ Error MTCNN di {img_path}: {e}")
        return None

    if face is None:
        # tidak ada wajah terdeteksi
        # print(f"⚠️ Tidak ada wajah di {img_path}")
        return None

    # face shape: (3, 160, 160)
    return face


def extract_embedding_resnet(face_tensor):
    """
    Ambil embedding 2048-dim dari wajah menggunakan ResNet50.
    """
    if face_tensor is None:
        return None
    face_pil = to_pil_image(face_tensor)
    x = transform_resnet(face_pil).unsqueeze(0).to(device)  # (1,3,224,224)
    with torch.no_grad():
        emb = resnet(x)  # (1, 2048)
    emb = emb.view(-1).cpu().numpy()  # (2048,)
    return emb


# ================================
# 5. BUILD EMBEDDINGS DARI DATASET
# ================================
def build_embeddings(data_dir, extractor_fn, desc="Building embeddings"):
    """
    Loop semua folder orang di data_dir, baca semua foto valid,
    crop wajah, ambil embedding pakai extractor_fn, simpan embeddings + label.
    """
    from tqdm import tqdm

    embeddings = []
    labels = []

    persons = [
        p for p in os.listdir(data_dir)
        if os.path.isdir(os.path.join(data_dir, p))
    ]

    for person_name in tqdm(persons, desc=desc):
        person_folder = os.path.join(data_dir, person_name)

        for img_name in os.listdir(person_folder):
            if not img_name.lower().endswith(valid_ext):
                continue

            img_path = os.path.join(person_folder, img_name)

            # 1) crop face
            face = crop_face(img_path)
            if face is None:
                continue

            # 2) ambil embedding
            emb = extractor_fn(face)
            if emb is None:
                continue

            embeddings.append(emb)
            labels.append(person_name)

    embeddings = np.array(embeddings)
    print(f"✅ {desc} selesai. Shape embeddings: {embeddings.shape}, labels: {len(labels)}")
    return embeddings, labels


# ================================
# 6. TRAIN & EVAL RESNET50 + LOGREG
# ================================
print("\n🚀 Membuat embedding ResNet50...")
resnet_embeddings, resnet_labels = build_embeddings(
    data_dir, extract_embedding_resnet, desc="ResNet50 embeddings"
)

# Hitung jumlah sampel per kelas
label_counts_res = Counter(resnet_labels)
print("\n📌 Jumlah sampel per kelas (ResNet, sebelum filter):")
print(label_counts_res)

# Filter: buang kelas yang cuma punya < 2 sampel
filtered_embeddings_res = []
filtered_labels_res = []

for emb, lbl in zip(resnet_embeddings, resnet_labels):
    if label_counts_res[lbl] >= 2:
        filtered_embeddings_res.append(emb)
        filtered_labels_res.append(lbl)

filtered_embeddings_res = np.array(filtered_embeddings_res)

num_samples_res = len(filtered_labels_res)
num_classes_res = len(set(filtered_labels_res))

print("\n✅ Setelah filter kelas < 2 sampel (ResNet):")
print("Shape embeddings:", filtered_embeddings_res.shape)
print("Total label:", num_samples_res)
print("Total kelas:", num_classes_res)

# Hitung test_size minimal supaya n_test >= n_classes (syarat stratify)
min_test_size_res = num_classes_res / num_samples_res + 1e-6
test_size_res = max(0.2, min_test_size_res)

print(f"👉 test_size ResNet diset ke: {test_size_res:.3f}")

# Encode label ke angka 0..N-1
le_res = LabelEncoder()
y_encoded_res = le_res.fit_transform(filtered_labels_res)

X_train_res, X_val_res, y_train_res, y_val_res = train_test_split(
    filtered_embeddings_res,
    y_encoded_res,
    test_size=test_size_res,
    random_state=42,
    stratify=y_encoded_res
)

print("X_train_res:", X_train_res.shape, "X_val_res:", X_val_res.shape)


print("\n🧪 Training Logistic Regression di atas embedding ResNet50...")

clf_res = LogisticRegression(
    max_iter=5000,
    n_jobs=-1,
    multi_class="auto"
)
clf_res.fit(X_train_res, y_train_res)

y_pred_res = clf_res.predict(X_val_res)

acc_res = accuracy_score(y_val_res, y_pred_res)
prec_res, rec_res, f1_res, _ = precision_recall_fscore_support(
    y_val_res, y_pred_res, average="weighted", zero_division=0
)
cm_res = confusion_matrix(y_val_res, y_pred_res)

print("\n📊 Classification report (ResNet50 + LogisticRegression):")
print(classification_report(
    y_val_res, y_pred_res,
    target_names=le_res.classes_,
    zero_division=0
))

print(f"\n✅ Accuracy ResNet50 + LogReg: {acc_res:.4f}")
print(f"✅ Weighted F1: {f1_res:.4f}")

# Simpan model ke file .pkl
model_data_resnet_logreg = {
    "model_type": "ResNet50 + LogisticRegression",
    "label_encoder": le_res,
    "classifier": clf_res,
    "validation_results": {
        "accuracy": acc_res,
        "precision_weighted": prec_res,
        "recall_weighted": rec_res,
        "f1_weighted": f1_res,
        "confusion_matrix": cm_res,
    }
}

resnet_logreg_path = os.path.join(models_dir, "face_recognition_resnet50_logreg.pkl")
with open(resnet_logreg_path, "wb") as f:
    pickle.dump(model_data_resnet_logreg, f)

print(f"\n💾 Model ResNet50 + LogisticRegression disimpan di: {resnet_logreg_path}")


🖥️ Device: cuda
📁 Data dir: /content/dataset/Train

📸 Loading MTCNN...
✅ MTCNN loaded!

🧠 Loading ResNet50 (ImageNet pretrained)...
✅ ResNet50 loaded!

🚀 Membuat embedding ResNet50...


ResNet50 embeddings:  81%|████████▏ | 57/70 [01:41<00:31,  2.43s/it]

⚠️ Gagal buka gambar /content/dataset/Train/Martua Kevin A.M.H.Lubis/IMG_20241208_223409_653 - MARTUA KEVIN ANDREAS MUAL H LUBIS.jpg: cannot identify image file '/content/dataset/Train/Martua Kevin A.M.H.Lubis/IMG_20241208_223409_653 - MARTUA KEVIN ANDREAS MUAL H LUBIS.jpg'


ResNet50 embeddings:  91%|█████████▏| 64/70 [01:51<00:09,  1.55s/it]

⚠️ Gagal buka gambar /content/dataset/Train/Lois Novel E Gurning/foto 3 - LOIS NOVEL E. GURNING.png: cannot identify image file '/content/dataset/Train/Lois Novel E Gurning/foto 3 - LOIS NOVEL E. GURNING.png'
⚠️ Gagal buka gambar /content/dataset/Train/Lois Novel E Gurning/foto 4 - LOIS NOVEL E. GURNING.png: cannot identify image file '/content/dataset/Train/Lois Novel E Gurning/foto 4 - LOIS NOVEL E. GURNING.png'


ResNet50 embeddings: 100%|██████████| 70/70 [02:01<00:00,  1.73s/it]

✅ ResNet50 embeddings selesai. Shape embeddings: (243, 2048), labels: 243

📌 Jumlah sampel per kelas (ResNet, sebelum filter):
Counter({'Joshua Palti Sinaga': 5, 'Bezalel Samuel Manik': 4, 'hayyatul fajri': 4, 'Arkan Hariz Chandrawinata Liem': 4, 'Fajrul Ramadhana Aqsa': 4, 'Kenneth Austin Wijaya': 4, 'Zefanya Danovanta Tarigan': 4, 'JP. Rafi Radiktya Arkan. R. AZ': 4, 'Dyo Dwi Carol Bukit': 4, 'Ikhsannudin Lathief': 4, 'Joyapul Hanscalvin Panjaitan': 4, 'Fayyadh Abdillah': 4, 'Gabriella Natalya Rumapea': 4, 'Dwi Arthur Revangga': 4, 'Zaky Ahmad Makarim': 4, 'Royfran Roger Valentino': 4, 'Desty Ananta Purba': 4, 'Abu Bakar Siddiq Siregar': 4, 'Femmy Aprillia Putri': 4, 'Bayu Ega Ferdana': 4, 'Rahmat Aldi Nasda': 4, 'Ichsan Kuntadi Baskara': 4, 'Falih Dzakwan Zuhdi': 4, 'Randy Hendriyawan': 4, 'Joy Daniella V': 4, 'Garland Wijaya': 4, 'Reynaldi Cristian Simamora': 4, 'Zakhi algifari': 4, 'Ahmad Faqih Hasani': 4, 'Dimas Azi Rajab Aizar': 4, 'Freddy Harahap': 4, 'Alfajar': 4, 'Rayhan Fati


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



📊 Classification report (ResNet50 + LogisticRegression):
                                 precision    recall  f1-score   support

             Nasya Aulia Efendi       0.00      0.00      0.00         1
           Abraham Ganda Napitu       0.00      0.00      0.00         1
       Abu Bakar Siddiq Siregar       1.00      1.00      1.00         1
             Ahmad Faqih Hasani       0.00      0.00      0.00         1
                        Alfajar       0.00      0.00      0.00         1
            Alief Fathur Rahman       0.00      0.00      0.00         1
 Arkan Hariz Chandrawinata Liem       1.00      1.00      1.00         1
               Bayu Ega Ferdana       0.00      0.00      0.00         1
          Bayu Prameswara Haris       0.00      0.00      0.00         1
           Bezalel Samuel Manik       0.50      1.00      0.67         1
           Bintang Fikri Fauzan       0.00      0.00      0.00         1
              Boy Sandro Sigiro       0.00      0.00      0.00   